In [73]:
# default_exp transplier

# longtail

> API details.

In [79]:
#export
import os
from dateutil import parser
from collections import namedtuple

with BlockTracker("../data/00000000.txt",stanza="Mid file 2") as f:
	f(print)("hello")
	f(print)("world")
	f(capture)("hello","world")
	f(print)("human fish")
	f(waste_some_time)(100)
with BlockTracker("../data/00000000.txt",stanza="Mid file 2") as f:
	f(print)("block 2")
	f(waste_some_time)(300)


In [94]:
#export

Blameline = namedtuple("Blameline","hash date func sig line")

class Transpiler(object):

    def parseBlame(self,line):
        st = line.index(" (")
        et = line.index(") ")

        hash = line[:st]
        label = line[st+2:et-1]
        prog = line[et+2:]

        timestamp = " ".join(label.split(" ")[-4:])
        dt_obj = parser.parse(timestamp)


        items = prog.split("\t")
        if len(items) < 2:
            return None

        func = items[0]
        sig = items[1]
        sig = sig[:-1]

        return Blameline(hash=hash,date=timestamp,func=func,sig = sig,line = line)

    def postProcess(self,group,writer):
        #print("in the post",group,len(group))
        dates = [x.date for x in group]

        if len(group)>0:
            # using max() + lambda
            # to get tuple info. of maximum value tuple
            res = max(group, key = lambda i : i[1])

            # printing result
            #print ("The name with maximum score is : " + res)
            writer.write(f"\nwith BlockTracker(\"data/{res.hash}.txt\",stanza=\"Mid file 2\") as f:\n")
            for x in group:
                writer.write(f"\tf({x.func})({x.sig})\n")

    def parse(self):
        blame = []
        cmd = 'cd {path};git blame {fname}'.format(
                    path=self.root,
                    fname=self.file_name)
        with os.popen(cmd) as process:
            blame =  process.readlines()

        group = []
        with  open(self.outfile, 'w') as writer:
            writer.write("""
from longtail.engine import BlockTracker
from longtail.commonlib import waste_some_time, lastWrapper, capture

""")
            for x in blame:
                fields = self.parseBlame(x)
                if fields is None:
                    self.postProcess(group,writer)
                    group = []
                else:
                    group.append(fields)
            self.postProcess(group,writer)


    def __init__(self,  root,file_name,out_file):
        self.file_name = file_name
        self.root = root
        self.outfile = out_file


In [95]:
#hide
tr = Transpiler(".","test/src/basic.txt","test/transpile/basic.py")
tr.parse()

In [96]:
#hide
!python test/transpile/basic.py

hello
world
hello
world
human fish
block 2


hello
world
hello
world
human fish


,name,signature,value,start,total,batch,stanza
0,print,'hello',None,5642.74,0.000235,5642.74247,Mid file 2
1,print,'world',None,5642.74,0.000527,5642.74247,Mid file 2
2,capture,"'hello', 'world'",None,5642.74,0.000367,5642.74247,Mid file 2
3,print,'human fish',None,5642.74,0.000010,5642.74247,Mid file 2
4,waste_some_time,100,None,5642.74,0.266837,5642.74247,Mid file 2


hello
world
human fish


,name,signature,value,start,total,batch,stanza
0,capture,"'hello', 'world'",None,4860.93,0.000340,4860.931309,Mid file 2
1,print,'human fish',None,4860.93,0.000021,4860.931309,Mid file 2
2,waste_some_time,100,None,4860.93,0.264946,4860.931309,Mid file 2
